# Numerical Optimization (CS215300) Assignment 3
## Introduction
In this assignment, we expect you to be able to solve constrained optimization problem by Scipy library. We want you to apply two algorithms on the following problem as benchmark, survey the method used in these libraries, and analysis the behaviour of these algorithms.
The library document link: https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

## Task
1. Please solve the following problrem by using trust-constr method:
$$\begin{array}{lll}
        \min_{x_1,x_2} & f(x_1,x_2)=-x_1-x_2 \\
        \mbox{s.t. } & -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2 \le 0  \\
         & -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36 \le 0   \\
         & 0 \le x_1 \le 3 \\
         & 0 \le x_2 \le 4 \\
\end{array}$$
2. Please use COBYLA method to solve the same problem.
3. In your report, please read the paper cited in the libraries, which gives the details of the algorithms. Write a brief introduction of the algorithms, and compare their behaviours in this benchmark. You are not necessarily to read the original paper, other resourses (ex. slides from other schools or surveys) are also acceptable, please include the link or paper name in your reference if you referred other resources.
4. Please provide the Jacobian and Hessian function in matrix form in your report. Basic latex syntax is supported in Markdown.
5. Rename this notebook file with your student ID and upload it to eeclass platform. (ex. 107xxxxxx.ipynb)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

### Define objective function

In [3]:
def f(x):
    return (-x[0]-x[1])

### Define constraint functions and derivatives
Note: Please not use sparse matrices.

In [4]:
# TODO: derive and define the functions
def cons_f(x):
    
    return np.array([(-2.0)*x[0]**4 + 8*x[0]**3 + (-8)*x[0]**2 + x[1] - 2, (-4)*x[0]**4 + 32*x[0]**3 + (-88)*x[0]**2 + 96*x[0] + x[1] -36])
    
def cons_Jacobian(x):

    return np.array([[(-8.0)*x[0]**3 + 24*x[0]**2 - 16*x[0], 1], [(-16)*x[0]**3 + 96*x[0]**2 - 176*x[0] +96, 1]])

def cons_Hessian(x,v):
    c1_hess = np.array([[(-24.0)*x[0]**2 + 48*x[0]-16, 0], [0, 0]])
    c2_hess = np.array([[(-48)*x[0]**2 + 192*x[0]-176, 0], [0, 0]])
    return v[0]*c1_hess + v[1]*c2_hess

def grad(x):
    return np.array([-1.0, -1.0])

def hess(x):
    return np.array([[0.0, 0.0], [0.0, 0.0]])

# TODO: Insert the functions above into a NonlinearConstraint obeject
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 0, jac=cons_Jacobian, hess=cons_Hessian)

### Define the bounds 

In [5]:
# TODO: define the bounds
bounds = Bounds([0, 0], [3.0, 4.0])

### Call the minimize library

In [6]:
from scipy.optimize import minimize

In [7]:
x0 = np.array([0, 0])
res = minimize(f, x0, method='trust-constr', jac=grad, hess=hess,
               constraints=[nonlinear_constraint],bounds=bounds)

### Print out the result you get

In [8]:
print(res.x) 

[0.61157004 3.44188615]


### Apply COBYLA method

In [9]:
x0 = np.array([0, 0])

bounds=[[0,3],[0,4]]

#construct the bounds in the form of constraints
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)

cons.append( {'type': 'ineq',
         'fun': lambda x: -((-2.0)*x[0]**4 + 8*x[0]**3 + (-8)*x[0]**2 + x[1] - 2)})
cons.append( {'type': 'ineq',
         'fun': lambda x: -((-4)*x[0]**4 + 32*x[0]**3 + (-88)*x[0]**2 + 96*x[0] + x[1] -36)})

res = minimize(f, x0, method='COBYLA', constraints=cons)
print(res.x)

[0.61160344 3.44210482]


## Report


Gradient (Objective function) = $\begin{bmatrix}
-1 \\
-1
\end{bmatrix}
$

Hessian (objective function) = $\begin{bmatrix}
0 & 0 \\
0 & 0
\end{bmatrix}
$


Jacobian = $\begin{bmatrix}
-8x_1^3+24x_1^2-16x_1 & 1 \\
-16x_1^3+96x_1^2-176x_1+96 & 1
\end{bmatrix}
$


Hessian (C1) = $\begin{bmatrix}
-24x_1^2+48x_1-16 & 0 \\
0 & 0
\end{bmatrix}
$


Hessian (C2) = $\begin{bmatrix}
-48x_1^2+192x_1-176 & 0 \\
0 & 0
\end{bmatrix}
$

**Details of the Algorithms (Brief Review) + Comparison:**

Method implementation **trust-constr** based on EQSQP for problems with constraints of the equality type and on TRIP for problems with constraints in the form of inequalities. Both methods are implemented by local minimum search algorithms in the confidence domain and are well suited for large-scale tasks.


A trust region is a subset of the objective function's region that is approximated using a model function in mathematical optimization (often a quadratic). The region is increased if a sufficient model of the objective function is found within the trust region; conversely, the region is constricted if the approximation is bad. The fit is determined by comparing the expected improvement from the model approximation to the actual improvement in the objective function. The criterion for expansion and contraction is simple ratio thresholding—a model function is only "trusted" in the region where it gives a decent approximation.

   
**COBYLA**, constrained optimization by linear approximation, is a derivation free optimizer that does not account for bounds explicitly. This works by creating a linear approximate of the objective and constraints for n+1 points in the design space that a simplex can work over to find the optimum in the trust region. The trust region radius is then modified as the algorithm moves towards the optimum. COBYLA takes significantly more function calls to get a worse resulting optimum. This is not unexpected with it implementing constraints by linear approximation. COBYLA does not allow for bounds for design variables to directly specified in the formulation. In other words, COBYLA does not satisfy the constraints and creates an infeasible design. To account for the bounds, we added additional constraints based on TA suggestions from stackoverflow.

All in all, since trust-costr method used first-order and second-order derivative information, it seems this method works well in this problem. Even if computing hessian is not very straightforward, SciPy allowed us to approximate hessian matrix in different methods such as SR1 and BFGS. The benefit of COBYLA is a derivation free and no need to compute second order derivation. However, since it creates a linear approximate of objective function and constraints, sometimes the performance of this algorithm degrades significantly as described above.


**Details about Implementation:**

First, we need to define our objective function and this work is straight-forward.

For trust-constr method, implementation is easy for bounds constraints since the SciPy optimizer for trust-constr method can consider the bound constraints implicitly and we can feed our bounds constraints in SciPy optimizer. For nonlinear constraint, we need do define hessian matrices for each constraints. In my implementation, I refered to each constraints as c1_hess and c2_hess. Moreover, we also need to define jacobian of constraints. After defining all required matrices, then we can pass our constraints parameters into "NonlinearConstraint" function. In this step, we should also define the inequality constraints bounds. In our case, it is -infinity to 0. Then, after choosing proper intial guess, we can see that the problem can be solved very well.


For COBYLA method, we have the same story for the objective function but for constraints, the story is a bit different. First of all, COBYLA cannot consider bounds for each variables implicity. For this reason, we need to define additional constraints for this issue. Based on stackoverflow, I define additional constraints to consider the bounds and it seems the successful when I run the code. Moreover, for other constraints such as nonlinear constraints and inequality, we need to define the constraints by dictionary based on SciPy doc. Actually for SLSQP and COBYLA, we have to define the constraints in this way. It is worth mentioning that, in SciPy, inequallity constraints considered as greater or equal than zero by default. So, we need to put minus before our inequality constraints to satisfy our problem since the inequality constraints in the problem are less or equal than zero. After running the code, we can see that this algorithm can find a solution suitably and also the answers are satisfying the constraints and bounds.





**Some observations from the initial guess:**

Because the objective function is high-dimensional, one or more suboptimal points may exist. The outcome is influenced by the initial guess and algorithm used. For example, for [0,0], we got [0.61157004 3.44188615] but for [1,0.5], we got [1.59964935 2.82000787] for trust-constr method. The story is also the same for COBYLA method. The remedy of this issue is using grid search or printing the point value and decide which is lower. 




**References:**

[1] Papers cited in Library

[2] SciPy Optimizer Doc

[3] Sorensen, D. C. (1982). "Newton's Method with a Model Trust Region Modification". SIAM J. Numer. Anal. 19 (2): 409–426.

[4] Forum discussion of EEClass